In [1]:
# Import python libraries 
import matplotlib as mpl
import matplotlib.pyplot as plt 

from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix

from dataclasses import dataclass 

# User-defined modules
from financial_data import *

In [2]:
# Loading datasets in order: 
# account, card, client, disp, district, loan, order, trans
data = load_original_dataset()
clean_data = load_clean_dataset()
clean_numerica_data = load_clean_numerical_dataset()